In [ ]:
pip install sae_lens

In [ ]:
import os
import numpy as np
import torch as t
from sae_lens import (
    SAE,
    ActivationsStore,
    HookedSAETransformer,
    LanguageModelSAERunnerConfig,
    SAEConfig,
    SAETrainingRunner,
    upload_saes_to_huggingface,
)
import gc

In [ ]:
device = "cpu"
t.set_grad_enabled(False)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
english_prompts = []
spanish_prompts = []
# Read and parse file
np.random.seed(42)
sampled_indices = np.random.choice(141543, size=1000, replace=False)
with open("/workspace/spa.txt", "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i in sampled_indices:
            parts = line.strip().split("\t")  # Split by tab
            if len(parts) >= 2:  # Ensure valid format
                english_prompts.append(parts[0])   # English sentence
                spanish_prompts.append(parts[1])   # Spanish translation

# Check first few examples
print(len(english_prompts))
print("English:", english_prompts[:5])
print("Spanish:", spanish_prompts[:5])

In [ ]:
gemma_9b: HookedSAETransformer = HookedSAETransformer.from_pretrained("gemma-2-9b", device=device)

sae_release = "gemma-scope-9b-pt-res-canonical"
sae_id = "layer_0/width_16k/canonical"

blobs_path = "/root/.cache/huggingface/hub/models--google--gemma-scope-9b-pt-res/blobs"
for i in range(42):
  sae, cfg_dict_layer, sparsity_layer = SAE.from_pretrained(
      release=sae_release,
      sae_id=f"layer_{i}/width_16k/canonical",
      device=str(device),
  )
  del cfg_dict_layer, sparsity_layer
  for j in range(10):
      _, cache = gemma_9b.run_with_cache_with_saes(spanish_prompts[100*j:100*(j+1)], saes=[sae])
      sae_acts_post = cache[f"{sae.cfg.hook_name}.hook_sae_acts_post"]
      t.save(sae_acts_post, f"/workspace/sae_acts/spanish_layer{i}_idx{(1+j)*100}.pt")

      del cache, sae_acts_post
      gc.collect()
      for item in os.listdir(blobs_path):
          item_path = os.path.join(blobs_path, item)
          os.remove(item_path)

      _, cache = gemma_9b.run_with_cache_with_saes(english_prompts[100*j:100*(j+1)], saes=[sae])
      sae_acts_post = cache[f"{sae.cfg.hook_name}.hook_sae_acts_post"]
      t.save(sae_acts_post, f"/workspace/sae_acts/english_layer{i}_idx{(1+j)*100}.pt")

      del cache, sae_acts_post
      gc.collect()
      for item in os.listdir(blobs_path):
          item_path = os.path.join(blobs_path, item)
          os.remove(item_path)
  del sae
  gc.collect()
  print(i)